In [20]:
#from box.exceptions import BoxValueError
d = {"key": "value", "key1": "value1","key2": "value2"}

In [21]:
d["key1"]

'value1'

In [22]:
d.keys

<function dict.keys>

In [23]:
from box import ConfigBox
d2=ConfigBox({"key": "value", "key1": "value1","key2": "value2"})
d2

ConfigBox({'key': 'value', 'key1': 'value1', 'key2': 'value2'})

In [24]:
d2.key2

'value2'

In [25]:
from ensure import ensure_annotations
@ensure_annotations
def get_predict(x:int, y:int):
    return x*y

z=get_predict(2,"y")

EnsureError: Argument y of type <class 'str'> to <function get_predict at 0x000002AE80505630> does not match annotation type <class 'int'>

In [26]:
%pwd

't:\\Chicken_Disease_Classification'

In [1]:
import os
os.chdir("../")


In [2]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir : Path
    source_URL : str
    local_data_file : Path
    unzip_dir : Path

In [3]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories

In [4]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config

In [5]:
#Updating the components
import os
import urllib.request as  request
import zipfile
from cnnClassifier import logger
from cnnClassifier.utils.common import get_size

In [6]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config


    
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")  


    
    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [8]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2023-08-16 21:44:56,279: INFO: common: yaml file: T:\Chicken_Disease_Classification\config\config.yaml loaded successfully]
[2023-08-16 21:44:56,284: INFO: common: yaml file: params.yaml loaded successfully]
[2023-08-16 21:44:56,287: INFO: common: created directory at: artifacts]
[2023-08-16 21:44:56,288: INFO: common: created directory at: artifacts/data_ingestion]
[2023-08-16 21:45:01,555: INFO: 4037744573: artifacts/data_ingestion/data.zip download! with following info: 
Connection: close
Content-Length: 11616915
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "adf745abc03891fe493c3be264ec012691fe3fa21d861f35a27edbe6d86a76b1"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: 81CE:3BE2A8:1297C0:1A78A7:64DCF604
Accept-Ranges: bytes
Date: Wed, 16 Aug 2023 16:15:02 GMT
Via: 1.1 varnish
X-Serve